In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({"figure.dpi": 140})

In [ ]:
def u_solid_body(xy: np.ndarray, Omega: float) -> np.ndarray:
    """2D solid-body rotation about +z: u = Omega x r = Omega * (-y, x)."""
    x = xy[..., 0]
    y = xy[..., 1]
    return np.stack([-Omega * y, Omega * x], axis=-1)

def circle_points(R: float, n: int, angle_map=None) -> np.ndarray:
    """Closed loop points on a circle. angle_map maps s in [0,1) -> angle in [0,2pi)."""
    s = np.linspace(0.0, 1.0, n, endpoint=False)
    if angle_map is None:
        theta = 2 * np.pi * s
    else:
        theta = angle_map(s)
    return np.stack([R * np.cos(theta), R * np.sin(theta)], axis=-1)

def loop_line_integral_u_dot_dl(points: np.ndarray, Omega: float) -> float:
    """Discrete line integral ∮ u·dl around a closed polygonal chain given by points."""
    u = u_solid_body(points, Omega)
    dp = np.roll(points, -1, axis=0) - points
    return float(np.sum(np.einsum('ij,ij->i', u, dp)))

def signed_area(points: np.ndarray) -> float:
    x = points[:, 0]
    y = points[:, 1]
    return 0.5 * float(np.sum(x * np.roll(y, -1) - np.roll(x, -1) * y))

In [ ]:
Omega = 1.0
R = 0.2
n = 2000

# Different parameterizations (speed profiles) implemented as nonlinear angle maps
def uniform(s):
    return 2 * np.pi * s

def slow_fast(s):
    # Densify points on half the loop (non-uniform sampling)
    t = s + 0.25 * np.sin(2 * np.pi * s)
    t = (t - t.min()) / (t.max() - t.min())
    return 2 * np.pi * t

def clustered(s):
    # Cluster sampling near theta=0 (another non-uniform reparameterization)
    t = s**3
    return 2 * np.pi * t

maps = {"uniform": uniform, "slow_fast": slow_fast, "clustered": clustered}

rows = []
for name, amap in maps.items():
    pts = circle_points(R, n, angle_map=amap)
    circ = loop_line_integral_u_dot_dl(pts, Omega)
    A = signed_area(pts)
    rows.append((name, circ, A))

rows

In [ ]:
# Theory: ∮ u·dl = ∬ (∇×u)·dS = (2 Omega) * Area for solid-body rotation
A_theory = np.pi * R**2
circ_theory = 2 * Omega * A_theory

print(f"Theory area A = {A_theory:.6g}")
print(f"Theory circulation = 2*Omega*A = {circ_theory:.6g}")

for name, circ, A in rows:
    print(f"{name:10s}  area={A:.6g}  circ={circ:.6g}  rel.err={(circ-circ_theory)/circ_theory:+.3e}")

In [ ]:
# Map circulation to loop-phase response: Δθ = -κ α_eff ∮ u·dl
kappa = 1.0
alpha_eff = 1.0

dtheta = -kappa * alpha_eff * circ_theory
print(f"With κ={kappa}, α_eff={alpha_eff}, predicted Δθ ≈ {dtheta:.6g} rad (mod 2π)")

# Solve for κ α_eff needed to produce a target phase shift
target = 1.0  # rad
kappa_alpha_needed = abs(target / circ_theory)
print(f"To get |Δθ|≈{target} rad at Ω={Omega}, R={R}: need κ α_eff ≈ {kappa_alpha_needed:.6g} [units: s/m^2 if κ is dimensionless]")

In [ ]:
# Scaling with loop size: circulation ~ R^2, so required κ α_eff ~ 1/R^2 for fixed target
Rs = np.linspace(0.05, 0.5, 50)
circ = 2 * Omega * (np.pi * Rs**2)
need = 1.0 / circ

plt.figure(figsize=(5.2, 3.2))
plt.plot(Rs, need)
plt.xlabel('R (m)')
plt.ylabel('Required κ α_eff for |Δθ|=1 rad')
plt.title('Solid-body scaling: κ α_eff ∝ 1/R²')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()